In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torchvision import datasets, transforms, models
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import time
# import visualkeras
from ann_visualizer.visualize import ann_viz
from torchviz import make_dot
from torchsummaryX import summary


In [2]:
trainSet = pd.read_csv('Dataset/train.csv')
classes = trainSet[['id','species']].copy()
classes['id'] = classes['id'].astype(str)
classes['label'] = LabelEncoder().fit_transform(classes['species'])

image_folder = 'Dataset/images/'
imgs = []
labels = []
for i in sorted(os.listdir(image_folder)):
    id = i.split('.')[0]
    if id in classes['id'].values:
        labels.append(classes[classes['id'] == id]['label'].values[0])
        image = Image.open(os.path.join(image_folder, i)).convert('1')
        imgs.append(image)

print(f"There are {len(imgs)} images in the dataset")
print(f"There are {len(np.unique(labels))} labels in the dataset")

There are 990 images in the dataset
There are 99 labels in the dataset


In [3]:
X_train, X_test, y_train, y_test = train_test_split(imgs, labels, test_size=0.2, random_state=42, stratify=labels)
print(f"There are {len(X_train)} images in the training set")
print(f"There are {len(X_test)} images in the test set")
print(f"There are {len(np.unique(y_train))} classes in the training set")
print(f"There are {len(np.unique(y_test))} classes in the test set")

There are 792 images in the training set
There are 198 images in the test set
There are 99 classes in the training set
There are 99 classes in the test set


In [4]:
class CustomDataSet(torch.utils.data.Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image = self.transform(image=np.array(self.images[index],dtype=np.float32))['image']
        label = torch.tensor(self.labels[index], dtype=torch.long)
        return image, label


    
transform = A.Compose([
    A.Resize(224, 224),
    ToTensorV2()
])

trainDataSet = CustomDataSet(images=X_train, labels=y_train, transform=transform)
testDataSet = CustomDataSet(images=X_test, labels=y_test, transform=transform)



In [13]:
class CNNmodel(nn.Module):
  def __init__(self):
    super(CNNmodel,self).__init__()
    self.conv1= nn.Conv2d(in_channels=1,out_channels=64,kernel_size=3,padding=0)
    self.bn1= nn.BatchNorm2d(num_features=64)
    self.dropout = nn.Dropout(p=0.5)

    self.conv2= nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,padding=0)
    self.bn2= nn.BatchNorm2d(num_features=128)
    self.flatten = nn.Flatten()
    self.fc1 = nn.Linear(in_features=54*54*128,out_features=99)

  def forward(self,x):
    x =  F.max_pool2d(F.relu(self.bn1(self.conv1(x))),kernel_size=2,stride=2)
    x =  F.max_pool2d(F.relu(self.bn2(self.conv2(x))),kernel_size=2,stride=2)
    
    x = self.dropout(x)
    x = self.fc1(self.flatten(x))
    
    x = F.softmax(x, dim=1)

    return x


In [14]:
batch_size = 64
trainDataLoader = torch.utils.data.DataLoader(trainDataSet, batch_size=batch_size)
testDataLoader = torch.utils.data.DataLoader(testDataSet, batch_size=batch_size)

min_loss_epoch = 0
min_loss_value = -1
best_model_weights_paths = {}

best_val_loss = float('inf')  # Initialize with a large value
best_epoch = -1
best_model = None

train_losses = []
val_losses = []

criterion = nn.CrossEntropyLoss()
model = CNNmodel()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
num_epochs = 1000

writer = SummaryWriter('runs/cnn1'+time.strftime("%Y%m%d-%H%M%S"))

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in tqdm(trainDataLoader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch"):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_loss = running_loss / len(trainDataLoader) / batch_size
    train_losses.append(train_loss)
    
    # Validation loop
    model.eval()  
    correct_predictions = 0
    total_samples = 0
    with torch.no_grad():
        for images, labels in testDataLoader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()
    accuracy = correct_predictions / total_samples
    val_loss = 1-accuracy
    val_losses.append(val_loss)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_epoch = epoch
        best_model = model

    writer.add_scalar('Training Loss', train_loss, epoch)
    writer.add_scalar('Validation Loss', val_loss, epoch)
    

Epoch 7/1000:  31%|███       | 4/13 [00:26<01:00,  6.72s/batch]


KeyboardInterrupt: 